***
#### Import of required libraries

In [1]:
from traffic.core import Traffic
from tqdm.notebook import tqdm
from multiprocessing import Pool
import numpy as np
import pandas as pd

KeyboardInterrupt: 

***
#### Import data

In [ ]:
# t = Traffic.from_file(
#     "/mnt/beegfs/store/MIAR/merged/osn/takeoffs28_mass_may_22.parquet"
# )

# t = Traffic.from_file(
#     "/mnt/beegfs/store/MIAR/merged/osn/takeoffs28_mass_complete.parquet"
# )

t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/merged/osn/takeoffs28_route.parquet"
)

***
##### One-hot encoding

In [ ]:
route = t.data.route
t = Traffic(pd.get_dummies(data=t.data, columns=["route"], prefix="route"))
t.data["route"] = route

In [ ]:
t.data["route_DEGES"] = t.data["route_DEGES"].astype(int)
t.data["route_GERSA"] = t.data["route_GERSA"].astype(int)
t.data["route_VEBIT"] = t.data["route_VEBIT"].astype(int)
t.data["route_ZUE"] = t.data["route_ZUE"].astype(int)

***

In [ ]:
flightlist = []

for flight in tqdm(t):
    flightlist.append(flight)

In [ ]:
input_len = 10
output_len = 180

results = {}


def process_element(flight):
    temp = flight.data[
        [
            "latitude",
            "longitude",
            "altitude",
            "route_DEGES",
            "route_GERSA",
            "route_VEBIT",
            "route_ZUE",
        ]
    ]
    inputs = []
    outputs = []
    for i in range(len(temp) - input_len - output_len + 1):
        inputs.append(temp.iloc[i : i + input_len].to_numpy())
        outputs.append(
            temp.iloc[i + input_len : i + input_len + output_len][
                ["latitude", "longitude", "altitude"]
            ].to_numpy()
        )
    return inputs, outputs

In [ ]:
with Pool(processes=20) as pool:

    results = list(
        tqdm(pool.imap(process_element, flightlist), total=len(flightlist))
    )

pool.close()

input_listoflists, output_listoflists = zip(*results)

In [ ]:
flattened_input = [item for sublist in input_listoflists for item in sublist]
flattened_output = [item for sublist in output_listoflists for item in sublist]

In [ ]:
input_array = np.stack(flattened_input)
output_array = np.stack(flattened_output)

In [ ]:
# input_array.shape, output_array.shape

In [ ]:
np.save(
    "/home/krum/git/MT_krum_code/data/input_data_180_route.npy", input_array
)
np.save(
    "/home/krum/git/MT_krum_code/data/output_data_180_route.npy", output_array
)